# Indexação e Processamento de Consulta (Lab 4)  - Recuperação da Informação
**Aluno: Pedro Guedes Braga**


In [49]:
#coding: utf-8

import pandas as pd
import nltk
import operator
import math
import matplotlib.pyplot as pyplot
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import *
nltk.download('stopwords')
nltk.download('punkt') 
import csv
import time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**1 - Execute o algoritmo ilustrado na Fig. 5.8 do livro texto (pag. 157) para gerar um índice similar o mostrado na Fig. 5.4 (pag. 134). Guarde o índice em disco em formato csv. (1 pt).**




In [0]:
url_csv = "https://raw.githubusercontent.com/PedroGuedesBraga/ri_lab_01/master/output/results.csv"
df = pd.read_csv(url_csv)
df.dropna(inplace=True)  ##Retira campos NA (ex.: NaN, etc)
lista_de_noticias = df.text.unique() #Retorna uma lista onde cada elemento é o texto de uma noticia.
 # http://www.nltk.org/howto/portuguese_en.html - Tutorial StopWords
 # https://pythonspot.com/tokenizing-words-and-sentences-with-nltk/
palavras_a_ignorar = nltk.corpus.stopwords.words('portuguese')
palavras_a_ignorar.append('o')
palavras_a_ignorar.append('a')
palavras_a_ignorar.append('é')
palavras_a_ignorar.append('r')
  
special_chars = [',', '.', '-', '“', "”", ")", "(", ":", "%", "?", "$", "–", ""]



##Auxiliares
##Recebe um token(ja em lowercase) e aplica mais estratégias de tokenizacao, descritas pelas duas funcoes auxiliares acima.
def apply_tokenize_strategy(token):
  return remove_periods((remove_apostrophes(token)))

##Metodo auxiliar que remove apostrofos dos tokens, retornando um token em seu formato sem apostrofo
def remove_apostrophes(token):
  return token.replace("’", "").replace("'","")
  
##Metodo auxiliar que remove pontos em uma abreviacao
def remove_periods(token):
  return token.replace(".", "")

#Funcao auxiliar, que simplesmente resgata todos os documentos no indice
def getAllDocuments(index):
  list_of_documents = []
  for word in index:
    for tupla in index[word]:
      if(tupla[0] not in list_of_documents):
        list_of_documents.append(tupla[0])
        
  return list_of_documents




##Questao 1
##Funcao para construir o índice:
def build_index(documents):
  index = {}
  n = 0
  for document in documents:
    n = n + 1
    tokens = word_tokenize(document)
    for token in tokens:
      if((apply_tokenize_strategy(token.lower()) not in palavras_a_ignorar) and (apply_tokenize_strategy(token.lower()) not in special_chars) and ("," not in token.lower())):  ## Todos os tokens passam a serem tratados em LOWERCASE
        if(apply_tokenize_strategy(token.lower()) in index):
          index[apply_tokenize_strategy(token.lower())].append((n, tokens.count(token)))
        else:
          index[apply_tokenize_strategy(token.lower())] = [(n, tokens.count(token))]
  refactored_index = aux_refactor_index(index)
  return refactored_index


#funcao auxiliar responsavel por refatorar as listas invertidas do indice para retorna-la no formato pedido. - Remove duplicatas de documentos
def aux_refactor_index(index):
  for word in index:
    for tupla in index[word]:
      index[word] = set(index[word])
  return index
    

#Constrói o index
indice = build_index(lista_de_noticias)



#Formata os dados para serem escritos em um arquivo CSV
def data_to_write_in_csv(index):
  data_to_write = []
  for word in index :
    data_to_write.append(word + "," + (str(index[word])).replace(",",""))
  data_to_write_string = "\n".join(data_to_write)
  return data_to_write_string

##################################
########ESCREVE EM UM CSV#########
##################################
with open('index_result.csv', 'w') as writeFile:
  writeFile.write("Word,Inverted List\n")
  writeFile.write(data_to_write_in_csv(indice))
  

  
  


  
  

**2 - Implemente as abordagens de processamento de consulta documento-por-vez e termo-por-vez (Fig. 5.16 e 5.18). (2 pts)**

1. Defina 5 consultas de um termo somente.
2. Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).
3. Dê evidências de que sua implementação está correta.
4. Compare os tempos médios de execução e uso de memória de cada algoritmo.

In [0]:
##Questao 2
##Implemente as abordagens de processamento de consulta documento-por-vez e termo-por-vez (Fig. 5.16 e 5.18). (2 pts)
##print(indice)

#No caso, so sao retornados os documentos que tem score > 0

def documentAtTimeRetrieval(query, index, k):
  query = query.lower() #Como na indexacao (no processo de tokenizacao) foi feito downcase, na consulta deve ser feito tbm
  inverted_lists = []
  query_terms = query.split(" ")
  documents = getAllDocuments(index)
  document_scores = [] #Uma lista de tuplas no formato (score_do_documento, documento)
  
  #Adiciona em 'inverted_lists' as listas invertidas que correspondentes a cada termo.
  for term in query_terms: 
    if(term in index):
      inverted_lists.append(index[term])
  
  
  for document in documents:
    score_doc = 0
    for inverted_list in inverted_lists:
      for tupla in inverted_list:
        if(tupla[0] == document):
          score_doc = score_doc + tupla[1]
    if(score_doc > 0):
      document_scores.append((score_doc, document))
  document_scores.sort(key = operator.itemgetter(0), reverse=True)
  docs_to_retrieve = document_scores[:k]
  return docs_to_retrieve
  

  
  
def termAtTimeRetrieval(query, index, k):
  query = query.lower() #Como na indexacao (no processo de tokenizacao) foi feito downcase, na consulta deve ser feito tbm
  hash_table = {}
  inverted_lists = []
  query_terms = query.split(" ")
  document_scores = [] #Uma lista de tuplas no formato (score_do_documento, documento)
  
  for term in query_terms:
    if term in index:
      inverted_lists.append(index[term])
  
  for inverted_list in inverted_lists:
    for post in inverted_list:
      if(post[0] in hash_table.keys()):
        hash_table[post[0]] = hash_table[post[0]] + post[1]
      else:
        hash_table[post[0]] = post[1]
  
  #Neste ultimo bloco de codigo:
  #Retornando valores na forma desejada (apenas por 'formatacao' do resultado - Lista de tuplas: [(doc, score)])
  for value in hash_table:
    document_scores.append((hash_table[value], value))
    
  document_scores.sort(key = operator.itemgetter(0), reverse=True)
  docs_to_retrieve = document_scores[:k]  
    
  return docs_to_retrieve



######################################
## Definindo consultas de um termo: ##
######################################
one_term_query = ["bolsonaro", "presidente", "lula", "golpe", "economia"]
    
    
  


**Executando as 5 consultas para o algoritmo Document At Time (Documento por vez)**

In [55]:
query_terms = []
query_results = []
time_results_document_at_time = []
for consulta in one_term_query:
  initial_time = time.time()
  result = documentAtTimeRetrieval(consulta, indice, 10)
  final_time = time.time()
  algorithm_time = final_time - initial_time
  query_terms.append(consulta)
  query_results.append(result)
  time_results_document_at_time.append(algorithm_time)
  
pd.DataFrame({
    'Termo consultado: ': query_terms,
    'Resultados da consulta: ': query_results,
    'Tempo de execucao do algoritmo: ': time_results_document_at_time
})
  

,Termo consultado:,Resultados da consulta:,Tempo de execucao do algoritmo:
0,bolsonaro,"[(11, 28), (8, 71), (8, 55), (7, 16), (7, 103)...",0.035900
1,presidente,"[(9, 16), (8, 8), (8, 28), (7, 97), (7, 121), ...",0.027811
2,lula,"[(19, 121), (17, 56), (15, 20), (11, 129), (8,...",0.027600
3,golpe,"[(5, 3), (4, 41), (4, 8), (3, 109), (2, 71), (...",0.026831
4,economia,"[(3, 58), (3, 8), (3, 80), (3, 84), (2, 23), (...",0.026815


**Executando as 5 consultas para o algoritmo Term At Time (Termo por vez)**

In [57]:
query_terms = []
query_results = []
time_results_term_at_time = []
for consulta in one_term_query:
  initial_time = time.time()
  result = termAtTimeRetrieval(consulta, indice, 10)
  final_time = time.time()
  algorithm_time = final_time - initial_time
  query_terms.append(consulta)
  query_results.append(result)
  time_results_term_at_time.append(algorithm_time)
  
pd.DataFrame({
    'Termo consultado: ': query_terms,
    'Resultados da consulta: ': query_results,
    'Tempo de execucao do algoritmo: ': time_results_term_at_time
})

,Termo consultado:,Resultados da consulta:,Tempo de execucao do algoritmo:
0,bolsonaro,"[(11, 28), (8, 71), (8, 55), (7, 16), (7, 124)...",0.000093
1,presidente,"[(9, 16), (8, 28), (8, 8), (7, 97), (7, 121), ...",0.000048
2,lula,"[(19, 121), (17, 56), (15, 20), (11, 129), (8,...",0.000028
3,golpe,"[(5, 3), (4, 41), (4, 8), (3, 109), (2, 71), (...",0.000021
4,economia,"[(3, 58), (3, 80), (3, 84), (3, 8), (2, 60), (...",0.000020


**EVIDENCIAS DE QUE A IMPLEMENTAÇÃO ESTÁ CORRETA**: 
  
  -    Como se sabe, o algoritmo termo-por-vez faz um maior uso de memória em relação ao documento-por-vez, porém em contrapartida o algoritmo termo-por-vez é mais rápido que o documento-por-vez, este último faz um menor uso de memória. Observando os tempos nas tabelas acima, pode-se evidenciar isso.
  
**COMPARANDO OS TEMPOS MEDIOS DE EXECUCAO E USO DE MEMORIA DE CADA ALGORITMO**

  - Sabe-se que o algoritmo termo-por-vez usa mais memória que o documento-por-vez, sendo assim, o primeiro mais rápido que o último. O tempo médio de execucao de cada algoritmo pode ser observado abaixo:

In [58]:
document_at_time_medium_time = sum(time_results_document_at_time)/5
term_at_time_medium_time = sum(time_results_term_at_time)/5

col1 = [document_at_time_medium_time]
col2 = [term_at_time_medium_time]

pd.DataFrame({
    'Tempo medio documento-por-vez: ': col1,
    'Tempo medio termo-por-vez: ': col2
})

,Tempo medio documento-por-vez:,Tempo medio termo-por-vez:
0,0.028991,0.000042


**3 - Implemente uma das versões de consulta conjuntiva (AND) (Fig. 5.20 ou 5.21). (2 pts)**

- Defina 5 consultas conjuntivas (AND).
- Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).
- Dê evidências de que sua implementação está correta.

In [59]:
def documentAtTimeRetrievalConjuctive(query, index, k):
  query = query.lower() #Como na indexacao (no processo de tokenizacao) foi feito downcase, na consulta deve ser feito tbm
  inverted_lists = []
  query_terms = query.split(" ")
  documents = getAllDocuments(index)
  document_scores = [] #Uma lista de tuplas no formato (score_do_documento, documento)
  
  #Adiciona em 'inverted_lists' as listas invertidas que correspondentes a cada termo.
  for term in query_terms: 
    if(term in index):
      inverted_lists.append(index[term])
  
  
  for document in documents:
    # -> Em cada documento, na medida que se acha um termo que está na query,
    # decrementa-se terms_found em 1 unidade. So add o documento no resultado se
    # terms_found == 0. Ou seja, o doc tem todos os termos
    terms_found = len(query_terms)
    score_doc = 0
    for inverted_list in inverted_lists:
      for tupla in inverted_list:
        if(tupla[0] == document):
          score_doc = score_doc + tupla[1]
          terms_found = terms_found - 1
    if(score_doc > 0):
      if(terms_found == 0):
        document_scores.append((score_doc, document))
  document_scores.sort(key = operator.itemgetter(0), reverse=True)
  docs_to_retrieve = document_scores[:k]
  return docs_to_retrieve

###############################################
#####Defina 5 consultas conjuntivas (AND).#####
###############################################
conjuctive_queries = ['bolsonaro mesa', 'brasil estados unidos', 'teste de memoria', 'consulta conjutiva', 'teste de consulta']

query_terms = []
query_results = []
time_results_doc_at_timeConjuctive = []
for consulta in conjuctive_queries:
  initial_time = time.time()
  result = documentAtTimeRetrievalConjuctive(consulta, indice, 10)
  final_time = time.time()
  algorithm_time = final_time - initial_time
  query_terms.append(consulta)
  query_results.append(result)
  time_results_doc_at_timeConjuctive.append(algorithm_time)
  
pd.DataFrame({
    'Termos': query_terms,
    'Results': query_results,
    'Time results': time_results_doc_at_timeConjuctive
})
  



,Termos,Results,Time results
0,bolsonaro mesa,"[(11, 28), (8, 71), (8, 55), (6, 88), (5, 115)...",0.040581
1,brasil estados unidos,"[(11, 8), (9, 16), (8, 78), (7, 137), (6, 108)...",0.028697
2,teste de memoria,[],0.027148
3,consulta conjutiva,[],0.026493
4,teste de consulta,[],0.026212
